In [1]:
from pathlib import Path
import yaml
import sys
import numpy as np
import pandas as pd
sys.path.append("..")
from ai4mat.models.megnet_pytorch.megnet_on_structures import MEGNetOnStructures

In [2]:
# model_name = "megnet_pytorch/sparse/05-12-2022_19-50-53/d6b7ce45"
model_name = "megnet_pytorch/sparse/pilot"
with open(Path("../trials/") / f"{model_name}.yaml", "r") as f:
    config = yaml.safe_load(f)

In [3]:
predictor = MEGNetOnStructures(config['model_params'])
predictor.load(Path("../datasets/checkpoints/combined_mixed_all_train/formation_energy_per_site") / f"{model_name}/0.pth")

/home/kna/.cache/pypoetry/virtualenvs/2d-defects-potential-learning-pYjw2mkT-py3.11/lib/python3.11/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'nn.glob.Set2Set' is deprecated, use 'nn.aggr.Set2Set' instead
  warnings.warn(out)


In [4]:
data = pd.read_pickle("../datasets/processed/low_density_defects/MoS2/data.pickle.gz")

In [5]:
predictor.predict_structures(data.defect_representation.iloc[0:10])

  0%|          | 0/10 [00:00<?, ?it/s]

array([[2.3063502 ],
       [3.1158915 ],
       [1.0325952 ],
       [1.0103437 ],
       [2.3063502 ],
       [1.0071394 ],
       [1.650349  ],
       [0.81523454],
       [4.6652026 ],
       [1.6376601 ]], dtype=float32)